In [1]:
import os,sys
sys.path.append('../')
import pickle
from DJSCCv2 import *
import torch
from functions import Imageset
from compressai.ops import compute_padding
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import warnings
from pytorch_msssim import ms_ssim
from tmp_functions import *
from tqdm import tqdm

In [3]:
Nt = 2
U = 2
torch.manual_seed(10)
H = torch.Tensor([[1,0.1],[0.3,1]])
P = [0.6846,0.2837]
# p_noise = [0.3**2,0.15**2]
p_noise = torch.Tensor([1,1])*1e-7
mode = "ZF"
W_dict = {
    "ZF":F.normalize(H@torch.inverse(H.T@H),dim=0),
    "OPT":torch.Tensor([[0.9976,-0.2448],[-0.0693,0.9696]]),
    "Random":F.normalize(torch.randn(Nt,U),dim=0),
    "MRC":F.normalize(H,dim=0)
}
sinr_downlink(W_dict,mode,P,H,p_noise)
mu_miso = CHN_MU_MISO(W_dict[mode],H,P,p_noise)


------------- ZF
interference_power:  tensor(2.5198e-16)
interference_power:  tensor(9.5007e-16)
sinr(linear):  [tensor(6377624.), tensor(2448929.5000)]
sinr(dB):  tensor([6.8047, 6.3890])


In [4]:
dev = 'cuda:7'
with open('../savedmodels/model_snrdb_10_bwr_0.120_2024_01_23_02_38_04.pth.tar','rb') as f:
    model0=pickle.load(f)
model = Autoencoder(6.8,0.120,(3,256,256))
model.load_state_dict(model0.state_dict())
model = model.to(dev)
dset = Imageset('../../compress_lip/Jianhao_datasets/CLIC/',transforms.Resize((256,256)),1,1,5)
dloaders = []
for u in range(U):
    dloaders.append(dataloader.DataLoader(dset,1,True))

Number of feature map:  12


In [35]:
torch.manual_seed(10)

perf_stats = [{"mse":RUN_STAT("mse"),"psnr":RUN_STAT("psnr"),"ssim":RUN_STAT("ssim")}
                 for _ in range(U)]
with torch.no_grad():
    for x in tqdm(zip(*dloaders),total=len(dloaders[0])):
        # x is the list of all users' images
        # next: process the loaded images for the encoder
        x = [xu.to(dev) for xu in x]
        outenc = [model.encoder(xu) for xu in x]
        # print("outenc: ", outenc[0].shape)
        outchn = mu_miso(outenc)
        # print("outchn: ", outchn.shape)
        xhat = [model.decoder(outchn_) for outchn_ in outchn]
        # print("outdec: ", xhat[0].shape)
        # plt.figure()
        # for u in range(len(x)):
        #     plt.subplot(int(f"2{U}{u+1}"))
        #     plt.imshow((x[u].squeeze().to('cpu')).permute(1,2,0))
        #     plt.subplot(int(f"2{U}{U+u+1}"))
        #     plt.imshow((xhat[u].squeeze().to('cpu')).permute(1,2,0))
        #     # mse = torch.mean((x[u])**2)
        #     # print("mse: ",u,mse)
        for xu,xhatu,perf_stat in zip(x,xhat,perf_stats):
            mse = torch.mean((xu-xhatu)**2)
            # print("mse: ",u,mse)
            mse = torch.mean((xu-xhatu)**2)
            psnr = 10*np.log10(1/mse.item())
            ssim = ms_ssim(xu,xhatu,data_range=1.)
            perf_stat["mse"](mse)
            perf_stat["psnr"](psnr)
            perf_stat["ssim"](ssim)
for u in range(U):
    for k,v in perf_stats[u].items():
        print(v)

100%|██████████| 5/5 [00:01<00:00,  4.19it/s]

[mse  ]: [N=5.00E+00] [EX:  3.826E-03] [VarX:  4.835E-06]
[psnr ]: [N=5.00E+00] [EX:  2.489E+01] [VarX:  6.526E+00]
[ssim ]: [N=5.00E+00] [EX:  9.785E-01] [VarX:  1.462E-04]
[mse  ]: [N=5.00E+00] [EX:  3.826E-03] [VarX:  4.835E-06]
[psnr ]: [N=5.00E+00] [EX:  2.489E+01] [VarX:  6.526E+00]
[ssim ]: [N=5.00E+00] [EX:  9.785E-01] [VarX:  1.464E-04]


In [21]:
torch.sum(outenc[0]**2)/outenc[0][0].numel(),outenc[0].shape

(tensor(0.5209, device='cuda:7'), torch.Size([1, 12, 64, 64]))

In [36]:
class RUN_STAT:
    def __init__(self,name=""):
        self.name = name
        self.num = 0
        self.EX = 0
        self.EX2 = 0
    def __call__(self,x):
        self.EX = (self.EX*self.num+x)/(self.num+1)
        self.EX2= (self.EX2*self.num+x**2)/(self.num+1)
        self.num += 1
    def __str__(self):
        var = self.EX2-(self.EX)**2
        if self.name:
            return f"[{self.name}]: [N={self.num:5.2E}] [EX: {self.EX:10.3E}] [VarX: {var:10.3E}]"
        else:
            return f"[N={self.num:5.2E}] [EX: {self.EX:10.3E}] [VarX: {var:10.3E}]"
a = RUN_STAT("my")
a(1)
print(a)

[my]: [N=1.00E+00] [EX:  1.000E+00] [VarX:  0.000E+00]


In [37]:
mse_stat = RUN_STAT("mse")
with torch.no_grad():
    for x in tqdm(dloaders[0]):
        # x is the list of all users' images
        # next: process the loaded images for the encoder
        x = x.to(dev)
        xhat = model(x)
        mse = torch.mean((x-xhat)**2)
        mse_stat(mse)
print(mse_stat)

100%|██████████| 5/5 [00:00<00:00,  8.37it/s]

[mse]: [N=5.00E+00] [EX:  5.012E-03] [VarX:  8.009E-06]
